<img src="../assets/header_notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>ESA - Black Sea Deoxygenation Emulator</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# ----------
# Librairies
# ----------
import os
import sys
import cv2
import dawgz
import wandb
import xarray
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Dawgz (jobs //)
from dawgz import job, schedule

# -------------------
# Librairies (Custom)
# -------------------
# Adding path to source folder to load custom modules
sys.path.insert(1, '../src/debs/')
sys.path.insert(1, '../scripts/')

# Loading libraries
from metrics              import *
from tools                import *
from dataset              import *
from dataloader           import *
from tools                import *

# -------
# Jupyter
# -------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to the .py directory
%cd ../src/debs/

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>SCRIPTS</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Training a neural network:
%run __training.py --config local

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# -----------------
#    Parameters
# -----------------
#
# Time window
month_starting = 1
month_ending   = 1
year_starting  = 1980
year_ending    = 1980

# Maximum depth observed for oxygen, what is left is masked [m] (Note: To observe only the continental shelf set it to ~200m).
depth_max_oxygen = 200

# ------------------
#  Loading the data
# ------------------
# Loading the different datasets
BSD_dataset = BlackSea_Dataset(year_start  = year_starting,
                               year_end    = year_ending,
                               month_start = month_starting,
                               month_end   = month_ending)

# Loading the days ID (used to give temporal information to the model)
days_ID = BSD_dataset.get_days()

# Loading the different inputs
data_temperature   = BSD_dataset.get_data(variable = "temperature", region = "surface")
data_salinity      = BSD_dataset.get_data(variable = "salinity",    region = "surface")
data_chlorophyll   = BSD_dataset.get_data(variable = "chlorophyll", region = "surface")
data_kshort        = BSD_dataset.get_data(variable = "kshort",      region = "surface")
data_klong         = BSD_dataset.get_data(variable = "klong",       region = "surface")

# Loading the output
data_oxygen        = BSD_dataset.get_data(variable = "oxygen",
                                          region   = "bottom",
                                          depth    = depth_max_oxygen)

# Loading spatial information
bathy = BSD_dataset.get_depth(unit = "meter")
mesh  = BSD_dataset.get_mesh(x = 258, y = 578)

# Loading the black sea masks
bs_mask             = BSD_dataset.get_mask(depth = None)
bs_mask_with_depth  = BSD_dataset.get_mask(depth = depth_max_oxygen)
bs_mask_complete    = get_complete_mask(data_oxygen, bs_mask_with_depth)

# -----------------------
#  Preprocessing the data
# -----------------------
#
# Croping the data to be a multiple of 2
data_temperature   = crop(data_temperature,   factor = 2)
data_salinity      = crop(data_salinity,      factor = 2)
data_chlorophyll   = crop(data_chlorophyll,   factor = 2)
data_kshort        = crop(data_kshort,        factor = 2)
data_klong         = crop(data_klong,         factor = 2)
data_oxygen        = crop(data_oxygen,        factor = 2)
mesh               = crop(mesh,               factor = 2)
bathy              = crop(bathy,              factor = 2)
bs_mask            = crop(bs_mask,            factor = 2)
bs_mask_with_depth = crop(bs_mask_with_depth, factor = 2)
bs_mask_complete   = crop(bs_mask_complete,   factor = 2)


# Creation of the dataloader
BSD_loader = BlackSea_Dataloader(x = [data_temperature, data_chlorophyll],
                                 y = data_oxygen,
                                 t = days_ID,
                              mesh = mesh,
                              mask = bs_mask,
                   mask_with_depth = bs_mask_with_depth,
                        bathymetry = bathy,
                        window_inp = 4,
                        window_out = 1,
                    window_transfo = 1,
                              mode = "regression")

# Retrieving the datasets
ds_train      = BSD_loader.get_dataloader("train")
ds_validation = BSD_loader.get_dataloader("validation")
ds_test       = BSD_loader.get_dataloader("test")